In [399]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [400]:
a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")

a = a[['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%']]
a[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%']].apply(pd.to_numeric, errors="ignore")
a = a.replace(to_replace='NO IDEA', value=0)
a = a.replace(to_replace='---', value=0)
a = a.fillna(0)
a = a.drop(a.loc[((a['games']>39) & (a['year']==2023)) | ((a['games']>40) & (a['year']==2022)) | 
   ((a['games']>33) & (a['year']==2021)) |  
   ((a['games']>35) & (a['year']==2020)) | 
   ((a['games']>40) & (a['year']==2019)) |
   (a['year']==0)]['Player Name'].index)
a.columns

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_58032/2331070129.py:1: DtypeWarning: Columns (10,13,16,19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")


Index(['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started',
       'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct',
       ...
       'year', 'Ht', 'Wt', 'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%'],
      dtype='object', length=111)

In [401]:
a[a['Player Name']=='cam-spencer-1']

,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots,DR%
734,cam-spencer-1,2019-20,LOYOLA (MD),Patriot,23,9,28.0,3.5,7.1,0.571,1.7,3.0,0.556,1.8,4.1,0.6,1.3,1.5,0.0,0.6,2.8,3.4,3.1,0.3,0.0,0.8,1.7,10.0,-7.23,34,8.0,14.0,5.0,9.0,0.556,3.0,5.0,0.0,1.0,0.0,65.0,6,4.0,4.0,1.0,0.0,2.0,19.0,5.0,10.1,2.4,4.3,2.6,5.8,1.9,2.2,4.9,4.5,0.5,0.1,1.2,2.4,14.4,7.0,14.2,3.4,6.0,3.6,8.2,2.6,3.1,6.9,6.3,0.7,0.1,1.7,3.3,20.2,131.1,110.3,18.0,0.643,0.617,0.577,0.215,229.0,2.6,11.2,7.1,21.6,0.7,0.2,9.6,15.5,2.0,0.3,2.3,0.142,2.7,-2.2,0.5,2020,76,205.0,22,8.0,49.5,128.5,15.5,18.5,11.2
735,cam-spencer-1,2020-21,LOYOLA (MD),Patriot,5,3,25.6,3.4,8.0,0.545,2.0,5.0,1.0,1.4,3.0,0.286,2.0,2.6,0.0,0.4,4.0,4.4,3.0,1.0,0.2,2.2,2.6,10.2,-3.47,35,6.0,11.0,4.0,4.0,1.0,2.0,7.0,0.0,0.0,2.0,20.0,4,3.0,3.0,1.0,1.0,3.0,14.0,5.3,12.5,3.1,7.8,2.2,4.7,3.1,4.1,6.9,4.7,1.6,0.3,3.4,4.1,15.9,7.8,18.3,4.6,11.4,3.2,6.8,4.6,5.9,10.0,6.8,2.3,0.5,5.0,5.9,23.3,105.2,98.7,17.0,0.552,0.513,0.375,0.325,52.0,1.8,18.4,10.1,22.7,2.3,0.9,19.2,22.6,0.2,0.2,0.4,0.134,1.3,1.3,2.5,2021,76,205.0,5,3.0,18.0,103.3,22.8,22.4,18.4
736,cam-spencer-1,2021-22,LOYOLA (MD),Patriot,30,30,36.9,6.9,14.8,0.636,5.0,9.2,0.75,2.0,5.6,0.571,3.0,3.5,1.0,0.5,4.3,4.8,3.2,2.3,0.2,1.3,1.5,18.9,-7.95,32,7.0,11.0,3.0,4.0,0.75,4.0,7.0,2.0,2.0,0.0,128.0,6,1.0,0.0,0.0,0.0,1.0,20.0,7.5,16.0,5.4,10.0,2.1,6.0,3.3,3.8,5.2,3.4,2.5,0.3,1.4,1.6,20.4,11.3,24.0,8.1,15.0,3.2,9.0,4.9,5.7,7.8,5.1,3.7,0.4,2.1,2.4,30.6,117.2,96.7,24.4,0.572,0.535,0.376,0.239,519.0,1.8,14.4,8.0,20.0,3.7,0.8,7.3,25.6,3.5,2.0,5.5,0.200,4.3,0.8,5.1,2022,76,207.0,29,29.0,92.2,115.7,24.6,28.8,14.4
737,cam-spencer-1,2022-23,RUTGERS,Big Ten,34,34,31.5,4.4,9.9,0.556,2.3,5.1,0.6,2.1,4.9,0.5,2.2,2.5,0.75,0.7,3.1,3.8,3.1,2.0,0.1,1.4,1.9,13.2,7.31,36,5.0,9.0,3.0,5.0,0.6,2.0,4.0,3.0,4.0,2.0,105.0,5,2.0,1.0,0.0,3.0,2.0,15.0,5.6,12.6,2.9,6.4,2.7,6.2,2.8,3.2,4.9,3.9,2.6,0.1,1.7,2.4,16.7,8.5,19.1,4.4,9.7,4.1,9.4,4.3,4.8,7.3,5.9,3.9,0.2,2.6,3.6,25.3,121.5,92.4,21.4,0.592,0.550,0.491,0.251,419.0,2.6,11.3,6.9,19.9,3.9,0.5,10.8,20.4,3.0,2.5,5.5,0.207,5.2,5.2,10.4,2023,76,207.0,34,34.0,78.2,119.4,19.8,21.5,11.3


In [402]:
beta_df = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/Prediction Names.csv")
beta_df = beta_df[['PLAYER_ID', 'KP Diff', 'Year Diff']]
beta_df


,PLAYER_ID,KP Diff,Year Diff
0,cam-spencer-1,0.96,1
1,dalton-knecht-1,16.87,1
2,josh-cohen-1,20.32,1
3,jordan-king-1,8.83,1
4,sean-bairstow-1,-4.16,1
5,bradley-ezewiro-1,-8.85,1
6,isaiah-carr-1,-7.98,1
7,demilade-adelekun-1,4.08,1
8,ogheneyole-akuwovo-1,12.33,1
9,steven-ashworth-1,4.69,1


In [403]:
beta_df = pd.merge(beta_df, a, left_on='PLAYER_ID', right_on='Player Name', how='inner')
beta_df = beta_df[beta_df['season']=='2022-23']
beta_df = beta_df.rename({'KP Diff': 'KP_Conf_Dif', 'Year Diff': 'Year_dif'}, axis=1)
beta_df['Calc A'] = beta_df['%Min']*beta_df['ws']
beta_df['Calc A2'] = beta_df['%Min']*beta_df['bpm']
beta_df['Calc A3'] = beta_df['%Min']*beta_df['ORtg']
list(beta_df.columns)


['PLAYER_ID',
 'KP_Conf_Dif',
 'Year_dif',
 'Player Name',
 'season',
 'Team',
 'conf_abbr',
 'games',
 'games_started',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg_pct',
 'fg2_per_g',
 'fg2a_per_g',
 'fg2_pct',
 'fg3_per_g',
 'fg3a_per_g',
 'fg3_pct',
 'ft_per_g',
 'fta_per_g',
 'ft_pct',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'sos',
 'mp',
 'fg',
 'fga',
 'fg2',
 'fg2a',
 'fg2_pct.1',
 'fg3',
 'fg3a',
 'ft',
 'fta',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'fg_per_min',
 'fga_per_min',
 'fg2_per_min',
 'fg2a_per_min',
 'fg3_per_min',
 'fg3a_per_min',
 'ft_per_min',
 'fta_per_min',
 'trb_per_min',
 'ast_per_min',
 'stl_per_min',
 'blk_per_min',
 'tov_per_min',
 'pf_per_min',
 'pts_per_min',
 'fg_per_poss',
 'fga_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_

In [404]:
beta_df['KP_Conf_Dif'] = beta_df['KP_Conf_Dif']*-1

beta_df['Calc B'] = (beta_df['stl_per_poss']+beta_df['blk_per_poss']+beta_df['DR%'])*beta_df['%Min']
beta_df['Calc C']=(beta_df['off_rtg']-beta_df['def_rtg'])*beta_df['%Min']

beta_df['Calc Overall A'] = beta_df['Calc A'] + beta_df['Calc B']
beta_df['Calc Overall A2'] = beta_df['Calc A2'] + beta_df['Calc B']
beta_df['Calc Overall A3'] = beta_df['Calc A3'] + beta_df['Calc B']

beta_df['player_type'] = 0


In [405]:
beta_df = beta_df.reset_index()
beta_df

,index,PLAYER_ID,KP_Conf_Dif,Year_dif,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots,DR%,Calc A,Calc A2,Calc A3,Calc B,Calc C,Calc Overall A,Calc Overall A2,Calc Overall A3,player_type
0,3,cam-spencer-1,-0.96,1,cam-spencer-1,2022-23,RUTGERS,Big Ten,34,34,31.5,4.4,9.9,0.556,2.3,5.1,0.6,2.1,4.9,0.5,2.2,2.5,0.75,0.7,3.1,3.8,3.1,2.0,0.1,1.4,1.9,13.2,7.31,36,5.0,9.0,3.0,5.0,0.6,2.0,4.0,3.0,4.0,2.0,105.0,5,2.0,1.0,0.0,3.0,2.0,15.0,5.6,12.6,2.9,6.4,2.7,6.2,2.8,3.2,4.9,3.9,2.6,0.1,1.7,2.4,16.7,8.5,19.1,4.4,9.7,4.1,9.4,4.3,4.8,7.3,5.9,3.9,0.2,2.6,3.6,25.3,121.5,92.4,21.4,0.592,0.550,0.491,0.251,419.0,2.6,11.3,6.9,19.9,3.9,0.5,10.8,20.4,3.0,2.5,5.5,0.207,5.2,5.2,10.4,2023,76,207.0,34,34.0,78.2,119.4,19.8,21.5,11.3,430.10,813.28,9337.08,1204.28,2275.62,1634.38,2017.56,10541.36,0
1,7,josh-cohen-1,-20.32,1,josh-cohen-1,2022-23,SAINT FRANCIS (PA),NEC,31,31,33.8,8.3,14.2,0.385,8.3,14.1,0.364,0.0,0.0,0.5,5.2,8.0,0.571,2.5,5.7,8.3,2.5,0.9,0.9,1.8,2.5,21.8,-9.27,34,5.0,13.0,4.0,11.0,0.364,1.0,2.0,4.0,7.0,6.0,177.0,9,3.0,2.0,0.0,0.0,2.0,15.0,9.8,16.7,9.8,16.7,0.0,0.0,6.2,9.4,9.8,2.9,1.1,1.0,2.1,3.0,25.8,14.2,24.2,14.2,24.1,0.0,0.1,8.9,13.6,14.1,4.2,1.5,1.5,3.0,4.3,37.2,124.3,104.6,30.5,0.608,0.585,0.002,0.563,648.0,9.2,19.5,14.5,18.0,1.5,2.7,9.0,30.0,4.8,1.3,6.0,0.230,5.9,-1.5,4.5,2023,82,220.0,29,29.0,86.2,118.4,29.9,30.0,19.3,517.20,387.90,10206.08,1922.26,1698.14,2439.46,2310.16,12128.34,0
2,11,jordan-king-1,-8.83,1,jordan-king-1,2022-23,ETSU,Southern,32,31,35.9,5.4,13.6,0.4,3.3,6.8,0.4,2.2,6.8,0.4,2.5,3.0,0.0,0.3,3.3,3.6,3.1,1.1,0.1,2.4,1.6,15.6,-4.44,36,4.0,10.0,2.0,5.0,0.4,2.0,5.0,0.0,0.0,0.0,104.0,6,3.0,0.0,0.0,3.0,0.0,10.0,6.1,15.2,3.7,7.6,2.4,7.6,2.8,3.3,4.0,3.5,1.2,0.1,2.6,1.8,17.3,9.1,22.8,5.5,11.3,3.6,11.4,4.2,5.0,6.0,5.2,1.8,0.2,4.0,2.7,26.0,104.9,106.9,15.4,0.518,0.478,0.502,0.218,477.0,1.1,10.6,5.9,17.9,1.8,0.3,13.6,25.0,1.8,1.1,2.9,0.101,0.5,-1.6,-1.1,2023,72,175.0,30,29.0,90.4,104.0,24.3,26.9,10.3,262.16,-99.44,9401.60,1111.92,-180.80,1374.08,1012.48,10513.52,0
3,15,sean-bairstow-1,4.16,1,sean-bairstow-1,2022-23,UTAH STATE,MWC,35,35,31.6,4.2,8.9,0.611,3.3,6.6,0.647,0.9,2.4,0.0,1.0,1.8,0.0,1.2,3.9,5.2,2.6,0.4,0.2,1.5,2.6,10.3,6.81,33,11.0,18.0,11.0,17.0,0.647,0.0,1.0,0.0,0.0,3.0,138.0,7,5.0,1.0,0.0,2.0,2.0,22.0,5.4,11.3,4.2,8.3,1.2,3.0,1.2,2.2,6.5,3.3,0.5,0.3,1.8,3.3,13.1,7.7,16.4,6.1,12.0,1.7,4.3,1.8,3.2,9.5,4.8,0.8,0.4,2.7,4.8,18.9,108.8,105.0,13.0,0.529,0.524,0.265,0.198,374.0,5.0,14.1,9.8,15.4,0.8,0.7,13.0,18.4,1.7,1.3,3.0,0.107,1.4,1.2,2.6,2023,80,205.0,34,34.0,78.6,105.7,18.2,20.1,14.0,235.80,204.36,8308.02,1194.72,298.68,1430.52,1399.08,9502.74,0
4,17,bradley-ezewiro-1,8.85,1,bradley-ezewiro-1,2022-23,GEORGETOWN,Big East,27,7,12.1,1.5,2.9,0.538,1.5,2.9,0.538,0.0,0.0,0.0,1.3,2.1,0.667,1.2,2.3,3.5,0.1,0.3,0.7,0.5,1.9,4.3,7.52,24,7.0,13.0,7.0,13.0,0.538,0.0,0.0,2.0,3.0,1.0,62.0,8,2.0,1.0,0.0,2.0,3.0,16.0,5.0,9.7,5.0,9.7,0.0,0.0,4.2,7.1,11.6,0.2,1.0,2.2,1.6,6.2,14.2,7.2,13.9,7.2,13.9,0.0,0.0,6.0,10.2,16.7,0.4,1.4,3.2,2.3,9.0,20.4,112.4,108.5,18.3,0.544,0.519,0.000,0.734,114.0,10.9,21.2,16.0,1.2,1.4,5.9,10.9,18.2,0.6,0.3,0.9,0.106,-0.5,0.5,0.0,2023,81,255.0,27,7

In [406]:
# All In One Go,
kp = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv')
#kp.head(10)
kp_conf_rat = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv')
kp_conf_rat = kp_conf_rat[['Year_Conf', 'Conf_Abb', 'Conf_Rk', 'Conf_Full', 'Conf_Rating']]
# Prep initial df,
d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace='NO IDEA', value=0)
df = df.replace(to_replace='---', value=0)
df = df.fillna(0)

# Convert to Numeric,
df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 
    'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 
    'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors='ignore')

# Add Contributions,
conditions = [(df['X.Poss_n1']>=28.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=24.0)& (df['X.Min_n1']>10.0), 
              (df['X.Poss_n1']>=20.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=16) & (df['X.Min_n1']>10.0),
            (df['X.Poss_n1']>=12.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']<12) & (df['X.Min_n1']>10.0), 
            (df['X.Min_n1']<10)]
#(df['X.Min_n1']<10),,
#values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer'],
values = [0, 1, 2, 3, 4, 5, 6]
df['Contribution_n1'] = np.select(conditions, values)

# Add Year in School KP,
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'),
            (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class'] = np.select(year_cond, year_val)

# Add Ht. Categories,
ht_cond = [(df['Ht']<=74), (df['Ht']>74) & (df['Ht']<=76), (df['Ht']>76) & (df['Ht']<=78),
   (df['Ht']>78) & (df['Ht']<=80), (df['Ht']>80)]

ht_val = [1, 2, 3, 4, 5]
df['Pos_Ht'] = np.select(ht_cond, ht_val)
#df[‘column’] =( df[‘column’] – df[‘column’].mean() ) / df[‘column’].std(),

    
# Additional Features + Model Prep,
df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>=0.1)].index)
df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>=0.1)].index)
#df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>0.1)].index)
#df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>0.1)].index)


df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
idx = df.groupby(['Team', 'year'])['games'].max()
a = pd.DataFrame(idx)
ab = a.reset_index()
ab.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab, left_on=['Team', 'year'], right_on=['Team_max', 'year_max'], how='left')
idx2 = df.groupby(['Team_n1', 'year_n1'])['games_n1'].max()
a2 = pd.DataFrame(idx2)
ab2 = a2.reset_index()
ab2.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab2, left_on=['Team_n1', 'year_n1'], right_on=['Team_max', 'year_max'], how='left')
df.columns
tt = df[['Player.Name', 'Team', 'year', 'games_max_x', 'Team_n1', 'year_n1', 'games_max_y', 'games_n1']]
tt[tt['Player.Name']=='tevin-mack-1']
#df = df.drop(df[(df['games']<=0.1*df['games_max_x'])].index)
#df = df.drop(df[(df['games_n1']<=0.1*df['games_max_y'])].index)
#df = df.drop(df.loc[(df['games_max_x']<15)].index)
#df = df.drop(df.loc[(df['games_max_y']<15)].index)

#df = df.drop(df.loc[(df['year']==2020) | (df['year_n1']==2020)].index)
df = df.drop(df[(df['X.Min_n1']<=10)].index)
df = df.drop(df[(df['X.Min']<=10)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*(df['ws']-min(df['ws']))
df['Calc A2'] = df['X.Min']*df['bpm']
# df['Calc A2'] = df['bpm']/(1-df['X.Min'])
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1']-min(df['ws_n1']))
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

# Calc_A_n1_cond = [(df['Calc A_n1']<=340), (df['Calc A_n1']>340) & (df['Calc A_n1']<=664), (df['Calc A_n1']>664) & (df['Calc A_n1']<=1010),,
#            (df['Calc A_n1']>1010) & (df['Calc A_n1']<=1415), (df['Calc A_n1']>1415)],

# Calc_A_n1_val = [0, 1, 2, 3, 4],
# df['Calc_A_Contr'] = np.select(Calc_A_n1_cond, Calc_A_n1_val),
column_to_test = 'Calc A2_n1'
# Calc_overall_n1_cond = [(df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()), ,
#                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()) & (df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()), ,
#                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()) & (df[column_to_test]<=8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()),,
#                         (df[column_to_test]>8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min())],

                        # & (df[column_to_test]<=4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min()), ,
                        #(df[column_to_test]>4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min())],
Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.1)),
                        (df[column_to_test]>df[column_to_test].quantile(0.1)) & (df[column_to_test]<=df[column_to_test].quantile(0.4)),
                        (df[column_to_test]>df[column_to_test].quantile(0.4)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
                        (df[column_to_test]>df[column_to_test].quantile(0.88))]
# Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.5)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.5)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.88))]
#Calc_A_n1_val = [2, 1, 0]
Calc_A_n1_val = [3, 2, 1, 0]
df['Calc_Overall_n1_Contr'] = np.select(Calc_overall_n1_cond, Calc_A_n1_val)

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_58032/4094357533.py:7: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_58032/4094357533.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Contribution_n1'] = np.select(conditions, values)
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_58032/4094357533.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [407]:
df[df['Player.Name']=='jordan-king-1']

,Unnamed: 0,player_type,Team,Team_n1,Unnamed..0_x,Player.Name,season,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,Yr,G,S,X.Min,ORtg,X.Poss,X.Shots,eFG.,TS.,OR.,DR.,ARate,TORate,Blk.,Stl.,FC.40,FD.40,FTRate,FTM.A,Pct,X2PM.A,Pct.1,X3PM.A,Pct.2,X.Pct,X.2PM.A,X.Pct.1,Unnamed..0_x_n1,Player.Name_n1,season_n1,conf_abbr_n1,games_n1,games_started_n1,mp_per_g_n1,fg_per_g_n1,fga_per_g_n1,fg_pct_n1,fg2_per_g_n1,fg2a_per_g_n1,fg2_pct_n1,fg3_per_g_n1,fg3a_per_g_n1,fg3_pct_n1,ft_per_g_n1,fta_per_g_n1,ft_pct_n1,orb_per_g_n1,drb_per_g_n1,trb_per_g_n1,ast_per_g_n1,stl_per_g_n1,blk_per_g_n1,tov_per_g_n1,pf_per_g_n1,pts_per_g_n1,sos_n1,mp_n1,fg_n1,fga_n1,fg2_n1,fg2a_n1,fg2_pct.1_n1,fg3_n1,fg3a_n1,ft_n1,fta_n1,orb_n1,drb_n1,trb_n1,ast_n1,stl_n1,blk_n1,tov_n1,pf_n1,pts_n1,fg_per_min_n1,fga_per_min_n1,fg2_per_min_n1,fg2a_per_min_n1,fg3_per_min_n1,fg3a_per_min_n1,ft_per_min_n1,fta_per_min_n1,trb_per_min_n1,ast_per_min_n1,stl_per_min_n1,blk_per_min_n1,tov_per_min_n1,pf_per_min_n1,pts_per_min_n1,fg_per_poss_n1,fga_per_poss_n1,fg2_per_poss_n1,fg2a_per_poss_n1,fg3_per_poss_n1,fg3a_per_poss_n1,ft_per_poss_n1,fta_per_poss_n1,trb_per_poss_n1,ast_per_poss_n1,stl_per_poss_n1,blk_per_poss_n1,tov_per_poss_n1,pf_per_poss_n1,pts_per_poss_n1,off_rtg_n1,def_rtg_n1,per_n1,ts_pct_n1,efg_pct_n1,fg3a_per_fga_pct_n1,fta_per_fga_pct_n1,pprod_n1,orb_pct_n1,drb_pct_n1,trb_pct_n1,ast_pct_n1,stl_pct_n1,blk_pct_n1,tov_pct_n1,usg_pct_n1,ows_n1,dws_n1,ws_n1,ws_per_40_n1,obpm_n1,dbpm_n1,bpm_n1,year_n1,Ht_n1,Wt_n1,Yr_n1,G_n1,S_n1,X.Min_n1,ORtg_n1,X.Poss_n1,X.Shots_n1,eFG._n1,TS._n1,OR._n1,DR._n1,ARate_n1,TORate_n1,Blk._n1,Stl._n1,FC.40_n1,FD.40_n1,FTRate_n1,FTM.A_n1,Pct_n1,X2PM.A_n1,Pct.1_n1,X3PM.A_n1,Pct.2_n1,X.Pct_n1,X.2PM.A_n1,X.Pct.1_n1,Season_x,TeamName_x,TeamName 2_x,Tempo_x,RankTempo_x,AdjTempo_x,RankAdjTempo_x,OE_x,RankOE_x,AdjOE_x,RankAdjOE_x,DE_x,RankDE_x,AdjDE_x,RankAdjDE_x,AdjEM_x,RankAdjEM_x,seed_x,Season_y,TeamName_y,TeamName 2_y,Tempo_y,RankTempo_y,AdjTempo_y,RankAdjTempo_y,OE_y,RankOE_y,AdjOE_y,RankAdjOE_y,DE_y,RankDE_y,AdjDE_y,RankAdjDE_y,AdjEM_y,RankAdjEM_y,seed_y,Season,TeamName,TeamName 2,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed,Year_Conf_x,Conf_Abb_x,Conf_Rk_x,Conf_Full_x,Conf_Rating_x,Year_Conf_y,Conf_Abb_y,Conf_Rk_y,Conf_Full_y,Conf_Rating_y,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating,Contribution_n1,Class,Pos_Ht,Team_max_x,year_max_x,games_max_x,Team_max_y,year_max_y,games_max_y,Year_dif,Team_AdjEM_dif,KP_Rk_Dif,KP_Conf_Dif,Calc A,Calc A2,Calc A3,Calc A2_n1,Calc A_n1,Calc A3_n1,Calc B,Calc B_n1,Calc C,Calc C_n1,Calc Overall A,Calc Overall A2,Calc Overall A3,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1,Calc_Overall_n1_Contr
4458,5678,1,SIENA,SIENA,2864.0,jordan-king-1,2019-20,MAAC,30,2,18.0,1.6,4.6,0.385,0.5,1.5,0.000,1.1,3.1,0.833,0.4,0.4,1.0,0.3,1.0,1.3,0.7,0.5,0.0,0.7,1.5,4.7,-5.66,36,5.0,13.0,0.0,7.0,0.000,5.0,6.0,1.0,1.0,0.0,30.0,5,2.0,1.0,0.0,3.0,0.0,16.0,3.5,10.2,1.0,3.4,2.5,6.8,0.8,0.9,2.8,1.6,1.0,0.1,1.6,3.3,10.4,5.3,15.2,1.5,5.1,3.7,10.1,1.2,1.3,4.2,2.3,1.5,0.1,2.3,4.8,15.5,99.5,105.1,8.0,0.491,0.471,0.667,0.087,129.0,1.8,6.4,4.2,7.1,1.5,0.2,12

In [408]:
pd.set_option('display.max_columns', None)
beta_df

,index,PLAYER_ID,KP_Conf_Dif,Year_dif,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots,DR%,Calc A,Calc A2,Calc A3,Calc B,Calc C,Calc Overall A,Calc Overall A2,Calc Overall A3,player_type
0,3,cam-spencer-1,-0.96,1,cam-spencer-1,2022-23,RUTGERS,Big Ten,34,34,31.5,4.4,9.9,0.556,2.3,5.1,0.6,2.1,4.9,0.5,2.2,2.5,0.75,0.7,3.1,3.8,3.1,2.0,0.1,1.4,1.9,13.2,7.31,36,5.0,9.0,3.0,5.0,0.6,2.0,4.0,3.0,4.0,2.0,105.0,5,2.0,1.0,0.0,3.0,2.0,15.0,5.6,12.6,2.9,6.4,2.7,6.2,2.8,3.2,4.9,3.9,2.6,0.1,1.7,2.4,16.7,8.5,19.1,4.4,9.7,4.1,9.4,4.3,4.8,7.3,5.9,3.9,0.2,2.6,3.6,25.3,121.5,92.4,21.4,0.592,0.550,0.491,0.251,419.0,2.6,11.3,6.9,19.9,3.9,0.5,10.8,20.4,3.0,2.5,5.5,0.207,5.2,5.2,10.4,2023,76,207.0,34,34.0,78.2,119.4,19.8,21.5,11.3,430.10,813.28,9337.08,1204.28,2275.62,1634.38,2017.56,10541.36,0
1,7,josh-cohen-1,-20.32,1,josh-cohen-1,2022-23,SAINT FRANCIS (PA),NEC,31,31,33.8,8.3,14.2,0.385,8.3,14.1,0.364,0.0,0.0,0.5,5.2,8.0,0.571,2.5,5.7,8.3,2.5,0.9,0.9,1.8,2.5,21.8,-9.27,34,5.0,13.0,4.0,11.0,0.364,1.0,2.0,4.0,7.0,6.0,177.0,9,3.0,2.0,0.0,0.0,2.0,15.0,9.8,16.7,9.8,16.7,0.0,0.0,6.2,9.4,9.8,2.9,1.1,1.0,2.1,3.0,25.8,14.2,24.2,14.2,24.1,0.0,0.1,8.9,13.6,14.1,4.2,1.5,1.5,3.0,4.3,37.2,124.3,104.6,30.5,0.608,0.585,0.002,0.563,648.0,9.2,19.5,14.5,18.0,1.5,2.7,9.0,30.0,4.8,1.3,6.0,0.230,5.9,-1.5,4.5,2023,82,220.0,29,29.0,86.2,118.4,29.9,30.0,19.3,517.20,387.90,10206.08,1922.26,1698.14,2439.46,2310.16,12128.34,0
2,11,jordan-king-1,-8.83,1,jordan-king-1,2022-23,ETSU,Southern,32,31,35.9,5.4,13.6,0.4,3.3,6.8,0.4,2.2,6.8,0.4,2.5,3.0,0.0,0.3,3.3,3.6,3.1,1.1,0.1,2.4,1.6,15.6,-4.44,36,4.0,10.0,2.0,5.0,0.4,2.0,5.0,0.0,0.0,0.0,104.0,6,3.0,0.0,0.0,3.0,0.0,10.0,6.1,15.2,3.7,7.6,2.4,7.6,2.8,3.3,4.0,3.5,1.2,0.1,2.6,1.8,17.3,9.1,22.8,5.5,11.3,3.6,11.4,4.2,5.0,6.0,5.2,1.8,0.2,4.0,2.7,26.0,104.9,106.9,15.4,0.518,0.478,0.502,0.218,477.0,1.1,10.6,5.9,17.9,1.8,0.3,13.6,25.0,1.8,1.1,2.9,0.101,0.5,-1.6,-1.1,2023,72,175.0,30,29.0,90.4,104.0,24.3,26.9,10.3,262.16,-99.44,9401.60,1111.92,-180.80,1374.08,1012.48,10513.52,0
3,15,sean-bairstow-1,4.16,1,sean-bairstow-1,2022-23,UTAH STATE,MWC,35,35,31.6,4.2,8.9,0.611,3.3,6.6,0.647,0.9,2.4,0.0,1.0,1.8,0.0,1.2,3.9,5.2,2.6,0.4,0.2,1.5,2.6,10.3,6.81,33,11.0,18.0,11.0,17.0,0.647,0.0,1.0,0.0,0.0,3.0,138.0,7,5.0,1.0,0.0,2.0,2.0,22.0,5.4,11.3,4.2,8.3,1.2,3.0,1.2,2.2,6.5,3.3,0.5,0.3,1.8,3.3,13.1,7.7,16.4,6.1,12.0,1.7,4.3,1.8,3.2,9.5,4.8,0.8,0.4,2.7,4.8,18.9,108.8,105.0,13.0,0.529,0.524,0.265,0.198,374.0,5.0,14.1,9.8,15.4,0.8,0.7,13.0,18.4,1.7,1.3,3.0,0.107,1.4,1.2,2.6,2023,80,205.0,34,34.0,78.6,105.7,18.2,20.1,14.0,235.80,204.36,8308.02,1194.72,298.68,1430.52,1399.08,9502.74,0
4,17,bradley-ezewiro-1,8.85,1,bradley-ezewiro-1,2022-23,GEORGETOWN,Big East,27,7,12.1,1.5,2.9,0.538,1.5,2.9,0.538,0.0,0.0,0.0,1.3,2.1,0.667,1.2,2.3,3.5,0.1,0.3,0.7,0.5,1.9,4.3,7.52,24,7.0,13.0,7.0,13.0,0.538,0.0,0.0,2.0,3.0,1.0,62.0,8,2.0,1.0,0.0,2.0,3.0,16.0,5.0,9.7,5.0,9.7,0.0,0.0,4.2,7.1,11.6,0.2,1.0,2.2,1.6,6.2,14.2,7.2,13.9,7.2,13.9,0.0,0.0,6.0,10.2,16.7,0.4,1.4,3.2,2.3,9.0,20.4,112.4,108.5,18.3,0.544,0.519,0.000,0.734,114.0,10.9,21.2,16.0,1.2,1.4,5.9,10.9,18.2,0.6,0.3,0.9,0.106,-0.5,0.5,0.0,2023,81,255.0,27,7

In [409]:
df_test1 = df[['bpm', 'Calc A2', 'ws', 
                 'KP_Conf_Dif', 'X.Poss', 'Year_dif',  'Calc A', 'Calc B', 'Calc A3', 
                 'DR.',  'Team_AdjEM_dif', 'player_type', 'Contribution_n1',
                 'Class', 'Calc C', 'ORtg',
                 'Calc_Overall_n1_Contr', 
                 'per', 'efg_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'usg_pct', 'tov_pct', 'Calc Overall A', 'Calc Overall A2', 'Calc Overall A3']]
seed = 7
test_size = 0.16
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
#model_test = GaussianNB()
model_test = GradientBoostingClassifier()
#X_temp = df_test1[keep_col]
#X_temp = df_test1[['player_type', 'Calc A2', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 'Calc A3']]
X_temp = df_test1[['player_type', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
#X_temp.head(10)

y_temp = df_test1[['Calc_Overall_n1_Contr']]
X_train_check, X_test_check, y_train_check, y_test_check = train_test_split(X_temp, y_temp, test_size=test_size, random_state=seed)

model_test.fit(X_train_check, y_train_check)
model_test.score(X_test_check, y_test_check)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6260720411663808

In [410]:
beta_X_prep = beta_df[['player_type', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
predictions = model_test.predict(X_test_check)
predictions

array([2, 1, 2, ..., 2, 1, 1])

In [411]:
beta_df

,index,PLAYER_ID,KP_Conf_Dif,Year_dif,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots,DR%,Calc A,Calc A2,Calc A3,Calc B,Calc C,Calc Overall A,Calc Overall A2,Calc Overall A3,player_type
0,3,cam-spencer-1,-0.96,1,cam-spencer-1,2022-23,RUTGERS,Big Ten,34,34,31.5,4.4,9.9,0.556,2.3,5.1,0.6,2.1,4.9,0.5,2.2,2.5,0.75,0.7,3.1,3.8,3.1,2.0,0.1,1.4,1.9,13.2,7.31,36,5.0,9.0,3.0,5.0,0.6,2.0,4.0,3.0,4.0,2.0,105.0,5,2.0,1.0,0.0,3.0,2.0,15.0,5.6,12.6,2.9,6.4,2.7,6.2,2.8,3.2,4.9,3.9,2.6,0.1,1.7,2.4,16.7,8.5,19.1,4.4,9.7,4.1,9.4,4.3,4.8,7.3,5.9,3.9,0.2,2.6,3.6,25.3,121.5,92.4,21.4,0.592,0.550,0.491,0.251,419.0,2.6,11.3,6.9,19.9,3.9,0.5,10.8,20.4,3.0,2.5,5.5,0.207,5.2,5.2,10.4,2023,76,207.0,34,34.0,78.2,119.4,19.8,21.5,11.3,430.10,813.28,9337.08,1204.28,2275.62,1634.38,2017.56,10541.36,0
1,7,josh-cohen-1,-20.32,1,josh-cohen-1,2022-23,SAINT FRANCIS (PA),NEC,31,31,33.8,8.3,14.2,0.385,8.3,14.1,0.364,0.0,0.0,0.5,5.2,8.0,0.571,2.5,5.7,8.3,2.5,0.9,0.9,1.8,2.5,21.8,-9.27,34,5.0,13.0,4.0,11.0,0.364,1.0,2.0,4.0,7.0,6.0,177.0,9,3.0,2.0,0.0,0.0,2.0,15.0,9.8,16.7,9.8,16.7,0.0,0.0,6.2,9.4,9.8,2.9,1.1,1.0,2.1,3.0,25.8,14.2,24.2,14.2,24.1,0.0,0.1,8.9,13.6,14.1,4.2,1.5,1.5,3.0,4.3,37.2,124.3,104.6,30.5,0.608,0.585,0.002,0.563,648.0,9.2,19.5,14.5,18.0,1.5,2.7,9.0,30.0,4.8,1.3,6.0,0.230,5.9,-1.5,4.5,2023,82,220.0,29,29.0,86.2,118.4,29.9,30.0,19.3,517.20,387.90,10206.08,1922.26,1698.14,2439.46,2310.16,12128.34,0
2,11,jordan-king-1,-8.83,1,jordan-king-1,2022-23,ETSU,Southern,32,31,35.9,5.4,13.6,0.4,3.3,6.8,0.4,2.2,6.8,0.4,2.5,3.0,0.0,0.3,3.3,3.6,3.1,1.1,0.1,2.4,1.6,15.6,-4.44,36,4.0,10.0,2.0,5.0,0.4,2.0,5.0,0.0,0.0,0.0,104.0,6,3.0,0.0,0.0,3.0,0.0,10.0,6.1,15.2,3.7,7.6,2.4,7.6,2.8,3.3,4.0,3.5,1.2,0.1,2.6,1.8,17.3,9.1,22.8,5.5,11.3,3.6,11.4,4.2,5.0,6.0,5.2,1.8,0.2,4.0,2.7,26.0,104.9,106.9,15.4,0.518,0.478,0.502,0.218,477.0,1.1,10.6,5.9,17.9,1.8,0.3,13.6,25.0,1.8,1.1,2.9,0.101,0.5,-1.6,-1.1,2023,72,175.0,30,29.0,90.4,104.0,24.3,26.9,10.3,262.16,-99.44,9401.60,1111.92,-180.80,1374.08,1012.48,10513.52,0
3,15,sean-bairstow-1,4.16,1,sean-bairstow-1,2022-23,UTAH STATE,MWC,35,35,31.6,4.2,8.9,0.611,3.3,6.6,0.647,0.9,2.4,0.0,1.0,1.8,0.0,1.2,3.9,5.2,2.6,0.4,0.2,1.5,2.6,10.3,6.81,33,11.0,18.0,11.0,17.0,0.647,0.0,1.0,0.0,0.0,3.0,138.0,7,5.0,1.0,0.0,2.0,2.0,22.0,5.4,11.3,4.2,8.3,1.2,3.0,1.2,2.2,6.5,3.3,0.5,0.3,1.8,3.3,13.1,7.7,16.4,6.1,12.0,1.7,4.3,1.8,3.2,9.5,4.8,0.8,0.4,2.7,4.8,18.9,108.8,105.0,13.0,0.529,0.524,0.265,0.198,374.0,5.0,14.1,9.8,15.4,0.8,0.7,13.0,18.4,1.7,1.3,3.0,0.107,1.4,1.2,2.6,2023,80,205.0,34,34.0,78.6,105.7,18.2,20.1,14.0,235.80,204.36,8308.02,1194.72,298.68,1430.52,1399.08,9502.74,0
4,17,bradley-ezewiro-1,8.85,1,bradley-ezewiro-1,2022-23,GEORGETOWN,Big East,27,7,12.1,1.5,2.9,0.538,1.5,2.9,0.538,0.0,0.0,0.0,1.3,2.1,0.667,1.2,2.3,3.5,0.1,0.3,0.7,0.5,1.9,4.3,7.52,24,7.0,13.0,7.0,13.0,0.538,0.0,0.0,2.0,3.0,1.0,62.0,8,2.0,1.0,0.0,2.0,3.0,16.0,5.0,9.7,5.0,9.7,0.0,0.0,4.2,7.1,11.6,0.2,1.0,2.2,1.6,6.2,14.2,7.2,13.9,7.2,13.9,0.0,0.0,6.0,10.2,16.7,0.4,1.4,3.2,2.3,9.0,20.4,112.4,108.5,18.3,0.544,0.519,0.000,0.734,114.0,10.9,21.2,16.0,1.2,1.4,5.9,10.9,18.2,0.6,0.3,0.9,0.106,-0.5,0.5,0.0,2023,81,255.0,27,7

In [412]:
pred = pd.DataFrame(predictions)
pd.set_option('display.max_columns', None)
#beta_df = y_test_check.reset_index()
pred = pd.merge(pred, beta_df, left_index=True, right_index=True)
pred

,0,index,PLAYER_ID,KP_Conf_Dif,Year_dif,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots,DR%,Calc A,Calc A2,Calc A3,Calc B,Calc C,Calc Overall A,Calc Overall A2,Calc Overall A3,player_type
0,2,3,cam-spencer-1,-0.96,1,cam-spencer-1,2022-23,RUTGERS,Big Ten,34,34,31.5,4.4,9.9,0.556,2.3,5.1,0.6,2.1,4.9,0.5,2.2,2.5,0.75,0.7,3.1,3.8,3.1,2.0,0.1,1.4,1.9,13.2,7.31,36,5.0,9.0,3.0,5.0,0.6,2.0,4.0,3.0,4.0,2.0,105.0,5,2.0,1.0,0.0,3.0,2.0,15.0,5.6,12.6,2.9,6.4,2.7,6.2,2.8,3.2,4.9,3.9,2.6,0.1,1.7,2.4,16.7,8.5,19.1,4.4,9.7,4.1,9.4,4.3,4.8,7.3,5.9,3.9,0.2,2.6,3.6,25.3,121.5,92.4,21.4,0.592,0.550,0.491,0.251,419.0,2.6,11.3,6.9,19.9,3.9,0.5,10.8,20.4,3.0,2.5,5.5,0.207,5.2,5.2,10.4,2023,76,207.0,34,34.0,78.2,119.4,19.8,21.5,11.3,430.10,813.28,9337.08,1204.28,2275.62,1634.38,2017.56,10541.36,0
1,1,7,josh-cohen-1,-20.32,1,josh-cohen-1,2022-23,SAINT FRANCIS (PA),NEC,31,31,33.8,8.3,14.2,0.385,8.3,14.1,0.364,0.0,0.0,0.5,5.2,8.0,0.571,2.5,5.7,8.3,2.5,0.9,0.9,1.8,2.5,21.8,-9.27,34,5.0,13.0,4.0,11.0,0.364,1.0,2.0,4.0,7.0,6.0,177.0,9,3.0,2.0,0.0,0.0,2.0,15.0,9.8,16.7,9.8,16.7,0.0,0.0,6.2,9.4,9.8,2.9,1.1,1.0,2.1,3.0,25.8,14.2,24.2,14.2,24.1,0.0,0.1,8.9,13.6,14.1,4.2,1.5,1.5,3.0,4.3,37.2,124.3,104.6,30.5,0.608,0.585,0.002,0.563,648.0,9.2,19.5,14.5,18.0,1.5,2.7,9.0,30.0,4.8,1.3,6.0,0.230,5.9,-1.5,4.5,2023,82,220.0,29,29.0,86.2,118.4,29.9,30.0,19.3,517.20,387.90,10206.08,1922.26,1698.14,2439.46,2310.16,12128.34,0
2,2,11,jordan-king-1,-8.83,1,jordan-king-1,2022-23,ETSU,Southern,32,31,35.9,5.4,13.6,0.4,3.3,6.8,0.4,2.2,6.8,0.4,2.5,3.0,0.0,0.3,3.3,3.6,3.1,1.1,0.1,2.4,1.6,15.6,-4.44,36,4.0,10.0,2.0,5.0,0.4,2.0,5.0,0.0,0.0,0.0,104.0,6,3.0,0.0,0.0,3.0,0.0,10.0,6.1,15.2,3.7,7.6,2.4,7.6,2.8,3.3,4.0,3.5,1.2,0.1,2.6,1.8,17.3,9.1,22.8,5.5,11.3,3.6,11.4,4.2,5.0,6.0,5.2,1.8,0.2,4.0,2.7,26.0,104.9,106.9,15.4,0.518,0.478,0.502,0.218,477.0,1.1,10.6,5.9,17.9,1.8,0.3,13.6,25.0,1.8,1.1,2.9,0.101,0.5,-1.6,-1.1,2023,72,175.0,30,29.0,90.4,104.0,24.3,26.9,10.3,262.16,-99.44,9401.60,1111.92,-180.80,1374.08,1012.48,10513.52,0
3,1,15,sean-bairstow-1,4.16,1,sean-bairstow-1,2022-23,UTAH STATE,MWC,35,35,31.6,4.2,8.9,0.611,3.3,6.6,0.647,0.9,2.4,0.0,1.0,1.8,0.0,1.2,3.9,5.2,2.6,0.4,0.2,1.5,2.6,10.3,6.81,33,11.0,18.0,11.0,17.0,0.647,0.0,1.0,0.0,0.0,3.0,138.0,7,5.0,1.0,0.0,2.0,2.0,22.0,5.4,11.3,4.2,8.3,1.2,3.0,1.2,2.2,6.5,3.3,0.5,0.3,1.8,3.3,13.1,7.7,16.4,6.1,12.0,1.7,4.3,1.8,3.2,9.5,4.8,0.8,0.4,2.7,4.8,18.9,108.8,105.0,13.0,0.529,0.524,0.265,0.198,374.0,5.0,14.1,9.8,15.4,0.8,0.7,13.0,18.4,1.7,1.3,3.0,0.107,1.4,1.2,2.6,2023,80,205.0,34,34.0,78.6,105.7,18.2,20.1,14.0,235.80,204.36,8308.02,1194.72,298.68,1430.52,1399.08,9502.74,0
4,1,17,bradley-ezewiro-1,8.85,1,bradley-ezewiro-1,2022-23,GEORGETOWN,Big East,27,7,12.1,1.5,2.9,0.538,1.5,2.9,0.538,0.0,0.0,0.0,1.3,2.1,0.667,1.2,2.3,3.5,0.1,0.3,0.7,0.5,1.9,4.3,7.52,24,7.0,13.0,7.0,13.0,0.538,0.0,0.0,2.0,3.0,1.0,62.0,8,2.0,1.0,0.0,2.0,3.0,16.0,5.0,9.7,5.0,9.7,0.0,0.0,4.2,7.1,11.6,0.2,1.0,2.2,1.6,6.2,14.2,7.2,13.9,7.2,13.9,0.0,0.0,6.0,10.2,16.7,0.4,1.4,3.2,2.3,9.0,20.4,112.4,108.5,18.3,0.544,0.519,0.000,0.734,114.0,10.9,21.2,16.0,1.2,1.4,5.9,10.9,18.2,0.6,0.3,0.9,0.106,-0.5,0.5,0.0,2023,8